In [1]:
import pandas as pd

In [2]:
COL_NAMES = '''rearrangement amino_acid frame_type
rearrangement_type templates reads frequency productive_frequency
cdr3_length v_family v_gene v_allele d_family d_gene d_allele j_family
j_gene j_allele v_deletions d5_deletions d3_deletions j_deletions
n2_insertions n1_insertions v_index n1_index n2_index d_index j_index
v_family_ties v_gene_ties v_allele_ties d_family_ties d_gene_ties
d_allele_ties j_family_ties j_gene_ties j_allele_ties sequence_tags
v_shm_count v_shm_indexes antibody sample_name species locus
product_subtype kit_pool total_templates productive_templates
outofframe_templates stop_templates dj_templates total_rearrangements
productive_rearrangements outofframe_rearrangements
stop_rearrangements dj_rearrangements total_reads
total_productive_reads total_outofframe_reads total_stop_reads
total_dj_reads productive_clonality productive_entropy
sample_clonality sample_entropy sample_amount_ng
sample_cells_mass_estimate fraction_productive_of_cells_mass_estimate
sample_cells fraction_productive_of_cells max_productive_frequency
max_frequency counting_method primer_set release_date sample_tags
fraction_productive order_name kit_id total_t_cells'''.split()

COLS_TO_LOAD = {
    'sample_name': str,
    'amino_acid': str,
    'j_family': str,
    'j_gene': str,
    'j_allele': 'Int64', # plain int does not allow NaN values (how Numpy represents nulls)
    'v_family': str,
    'v_gene': str,
    'v_allele': 'Int64'
}

def load_data_file(filename):
    """Load the required columns from a raw data file into a Pandas DataFrame."""
    return pd.read_csv(
        filename,
        sep='\t',
        na_values=['unresolved'],
        header=0, names=COL_NAMES, # replace existing column names
        index_col=False, # don't use the first column as the index
        usecols=COLS_TO_LOAD.keys(),
        dtype=COLS_TO_LOAD
    )

In [3]:
sample = load_data_file('misc/1mb_sample.tsv')
sample

,amino_acid,v_family,v_gene,v_allele,j_family,j_gene,j_allele,sample_name
0,CASSRTGPGNTIYF,TCRBV05,TCRBV05-04,1,TCRBJ01,TCRBJ01-03,1,HIP00110
1,CASSYRANTQGAGEAFF,TCRBV06,NaN,NaN,TCRBJ01,TCRBJ01-01,1,HIP00169
2,CASSPRNTGELFF,TCRBV03,NaN,NaN,TCRBJ02,TCRBJ02-02,1,HIP00594
3,CASSAITGVDSPLHF,TCRBV05,TCRBV05-05,1,TCRBJ01,TCRBJ01-06,1,HIP00602
4,NaN,TCRBV09,TCRBV09-01,NaN,TCRBJ02,TCRBJ02-02,1,HIP00614
...,...,...,...,...,...,...,...,...
924,NaN,TCRBV27,TCRBV27-01,1,TCRBJ01,TCRBJ01-04,1,Keck0119_MC1
925,CSATGFNTEAFF,TCRBV29,TCRBV29-01,1,TCRBJ01,TCRBJ01-01,1,Keck0120_MC1
926,CASRLGWGGTEAFF,TCRBV04,TCRBV04-02,1,TCRBJ01,TCRBJ01-01,1,Keck0120_MC1
927,CASSLRLTGELFF,TCRBV27,TCRBV27-01,1,TCRBJ02,TCRBJ02-02,1,Keck0120_MC1


In [4]:
from sqlalchemy import create_engine

In [5]:
def populate_db(engine, tablename, df):
    df.to_sql(tablename, engine, if_exists='append')

In [6]:
# Local test with sample data:
# engine = create_engine('postgresql://jacksonfellows@localhost:5432/postgres')
# populate_db(engine, 'test', sample)

To get the raw count:
```sql
SELECT COUNT(*) FROM hip_raw; => 161479578
```

I had to create a new index (PostgreSQL specific) before the SELECT DISTINCT would complete:
```sql
CREATE INDEX uniq_seq_idx ON hip_raw(amino_acid, j_family, j_gene, j_allele, v_family, v_gene, v_allele);
```
Create a table of unique sequences:
```sql
SELECT DISTINCT amino_acid, v_family, v_gene, v_allele, j_family, j_gene, j_allele INTO hip_uniq FROM hip_raw;
```

Number of unique sequences:
```sql
SELECT COUNT(*) FROM hip_uniq; => 94821206
```
The paper has a unique count of `89840865`.

**OLD WAY:**

Get a file with all of the HIP patients (`emerson-2017-natgen` is the directory with the unzipped data files):
```sh
ls emerson-2017-natgen | sed 's/\.tsv$//' | grep '^HIP' > hip_patients
```
Then load this file into the db (PostgreSQL specific):
```sql
CREATE TABLE hip_patients (
    sample_name text
);
COPY hip_patients FROM 'hip_patients';
SELECT COUNT(*) FROM hip_patients; => 666
```

In [3]:
COL_NAMES.index('sample_tags')

76

Get a file with all of the HIP patients and their CMV status
(`emerson-2017-natgen` is the directory with the unzipped data files):
```bash
for f in emerson-2017-natgen/HIP*.tsv; do printf '%s\t%s\n' "$(basename $f .tsv)" "$(sed '2q;d' $f | cut -f77 | sed 's/.*Virus Diseases:Cytomegalovirus \([-+]\).*/\1/;tx;s/.*/\\N/;q;:x;y/-+/01/')"; done > hip_patients
```
Then load this file into the db (PostgreSQL specific):
```sql
CREATE TABLE hip_patients (
    sample_name text PRIMARY KEY,
    cmv_status boolean
);
COPY hip_patients FROM 'hip_patients';
SELECT COUNT(*) FROM hip_patients; => 666
SELECT COUNT(*) FROM hip_patients WHERE cmv_status = TRUE; => 289
SELECT COUNT(*) FROM hip_patients WHERE cmv_status = FALSE; => 352
```

I just realized that my `v_allele` and `j_allele` columns were actually text not integers. This shouldn't be the case if you load things via the new method (explicit datatypes). Some of them were written with decimal points (e.g. `1.0`) so I couldn't just convert them to integers right away. Instead, I converted each column first to doubles (MAYBE: then to integers). PostgreSQL specific SQL:
```sql
ALTER TABLE hip_raw ALTER COLUMN j_allele TYPE double precision USING j_allele::double precision;
ALTER TABLE hip_raw ALTER COLUMN v_allele TYPE double precision USING v_allele::double precision;
```
After this, I had to recreate the `hip_uniq` table.
```sql
SELECT DISTINCT amino_acid, v_family, v_gene, v_allele, j_family, j_gene, j_allele INTO hip_uniq_2 FROM hip_raw; => SELECT 92329425
```
This is a smaller unique count than I got previously but still larger than the paper.

I think it makes sense to also index the unique table. (DOES IT???)
```sql
CREATE INDEX uniq_seq_idx_uniq_2 ON hip_uniq_2(amino_acid, j_family, j_gene, j_allele, v_family, v_gene, v_allele);
```

Hold up, the paper states that they only identify CMV-associated TCRs for the unique chains of the 641 subjects in cohort 1 with known CMV status. Up to now, my tables of unique chains have been for all patients.

Check that the count matches the paper:
```sql
SELECT COUNT(*) FROM hip_patients WHERE cmv_status IS NOT NULL; => 641
```

Join the raw sequences and patient status table and then filter by non-null CMV status:
```sql
SELECT b.sample_name, b.amino_acid, b.j_family, b.j_gene, b.j_allele, b.v_family, b.v_gene, b.v_allele INTO hip_raw_known_status FROM hip_patients a JOIN hip_raw b ON a.sample_name = b.sample_name WHERE a.cmv_status IS NOT NULL; => SELECT 156293009
```

Now I have to create another index and then do another `SELECT DISTINCT`.
```sql
CREATE INDEX uniq_seq_idx_known_status ON hip_raw_known_status(amino_acid, j_family, j_gene, j_allele, v_family, v_gene, v_allele);
SELECT DISTINCT amino_acid, v_family, v_gene, v_allele, j_family, j_gene, j_allele INTO hip_uniq_known_status FROM hip_raw_known_status; => SELECT 89709744
```

The paper has `89840865` unique sequences. This count is actually less than that, so I'm being more selective or the paper's count represent unique sequences from all (666) patients not just the (641) patients with known CMV status.

New approach: Create a table with of unique sequences that appear in more than one patient with the total number of patients they appear in and the number of positive patients they appear in.
```sql
SELECT SUM(cmv_status::int) AS positive, COUNT(cmv_status) AS total, amino_acid, j_family, j_gene, j_allele, v_family, v_gene, v_allele INTO hip_uniq_counts FROM hip_patients JOIN hip_raw ON hip_patients.sample_name = hip_raw.sample_name WHERE cmv_status IS NOT NULL GROUP BY amino_acid, j_family, j_gene, j_allele, v_family, v_gene, v_allele HAVING COUNT(cmv_status) > 1; => SELECT 12308047
```
That's actually wrong because there are sequences repeated *within* patients so sequences are counted multiple times for some patients. Let's make a table without intra-sequence repeats:
```sql
CREATE INDEX uniq_sample_seq_idx ON hip_raw(sample_name, amino_acid, j_family, j_gene, j_allele, v_family, v_gene, v_allele);
SELECT DISTINCT sample_name, amino_acid, v_family, v_gene, v_allele, j_family, j_gene, j_allele INTO hip_sample_seq_uniq FROM hip_raw; => SELECT 131306142
```
Now let's rerun that first query:
```sql
SELECT SUM(cmv_status::int) AS positive, COUNT(cmv_status) AS total, amino_acid, j_family, j_gene, j_allele, v_family, v_gene, v_allele INTO hip_uniq_counts FROM hip_patients JOIN hip_sample_seq_uniq ON hip_patients.sample_name = hip_sample_seq_uniq.sample_name WHERE cmv_status IS NOT NULL GROUP BY amino_acid, j_family, j_gene, j_allele, v_family, v_gene, v_allele HAVING COUNT(cmv_status) > 1; => SELECT 10801341
```

Ok, now we can run a Fisher's exact test on each row of that table and get a list of CMV-associated TCRs (I got 166, paper got 164). Let's count how many times each of those sequences appears in each patient. I don't really want to use the database and I don't know if there's an efficient way anyway.

Let's first extract only the fields we care about from the raw data:
```bash
time for f in emerson-2017-natgen/HIP*.tsv; do time cut -f2,10,11,12,16,17,18 $f > emerson-2017-natgen-tcr/$(basename $f); done
```

Now we need to make the files use the same format for nulls. I'm going with 'NA' (whatever R used when exporting the list of associated TCRs). The raw data files represent null with both an empty string ('') and 'unresolved'. While we're at it, let's switch to field seperator to ',' (also to match the list of associated TCRs).
```bash
time for f in emerson-2017-natgen-tcr/*; do time awk -F'\t' -i inplace 'BEGIN { OFS = ","; } function to_na(field) { if (field == "" || field == "unresolved") { return "NA"; } return field; } {print to_na($1),to_na($2),to_na($3),to_na($4),to_na($5),to_na($6),to_na($7)}' $f; done
```

Oops they should be CSV files now.
```bash
time for f in emerson-2017-natgen-tcr/*; do mv $f "${f%%.*}.csv"; done
```

Also, the order of the columns is different in these files than in the list of associated sequences.
```bash
awk -F',' -i inplace 'BEGIN {OFS=","} {print $1,$5,$6,$7,$2,$3,$4}' cmv_associated_tcrs.csv
```

NOTE: some of the allele numbers in these files have leading zeros (e.g. '01') and some don't (esp. the list of associated TCRs), so we need to process those fields as integers. Since the number of associated TCRs is so small compared to the number of rows I am comparing against, I feel like it makes sense to just stuff those in a hash table and iterate over the lines. I wrote a Python script to do this. Make sure to use unbuffered output ('-u') if I am going to pipe to tee.
```bash
python3 -u tools/find_associated_counts.py ../emerson-2017-natgen-tcr/* | tee counts.csv
```

Convert `hip_patients` to a CSV (and change nulls to 'NA').
```bash
awk -F'\t' 'BEGIN {OFS = ","} {print $1,$2 == "\\N" ? "NA" : $2}' hip_patients > hip_patients.csv
```

Join the two files on patient name.
```bash
join -t, -j1 <(sort hip_patients.csv) <(sort counts.csv) > hip_cmv_n_k.csv
```

```bash
for f in emerson-2017-natgen/*.tsv; do printf "%s,%s\n" "$(basename $f .tsv)" "$(sed '2q;d' $f | cut -f77 | sed -E 's/.*Inferred CMV status( \(trained on Cohort 1\))?: Inferred CMV ([-+]).*/\2/;tx;s/.*/NA/;q;:x;y/-+/01/')"; done > all_inferred
```